<a href="https://colab.research.google.com/github/tipiorgup/Tutorial_IDP_SAPs/blob/main/Simulation%20code/Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import os
import sys


In [20]:
!git clone https://github.com/tipiorgup/Tutorial_IDP_SAPs.git
!pip install openmm
!pip install MDAnalysis

fatal: destination path 'Tutorial_IDP_SAPs' already exists and is not an empty directory.


In [16]:
#Check in src the file parsers.py, it will show all options available

In [17]:
ratio='900-0'
ionic_strength='0.3'
peptide_pair='HHHEEEWWW-HWKHWKHWK'

In [19]:
!python clean_pair_code.py --ratio {ratio} \
    --ionic-strength {ionic_strength} \
    --ph 7.4 \
    --peptide-pair {peptide_pair} \
    --run-all \
    --residues-file /src/residues.csv \
    --steps 10000

Cannot import pdbfixer. This may affect 3SPN2 model.
Processing pair: HHHEEEWWW-HWKHWKHWK
Error processing pair HHHEEEWWW-HWKHWKHWK: [Errno 2] No such file or directory: '/src/residues.csv'
Error processing pair HHHEEEWWW-HWKHWKHWK: [Errno 2] No such file or directory: '/src/residues.csv'
